In [6]:
def GenerateData(jsons):
    list = []
    lastId = jsons[len(jsons) - 1]['id']

    for i in range(len(jsons)):
        result = jsons[i]

        if 'chifure' in result['title'] or 'chifure' in result['title']:
            date = datetime.strptime(result['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
            createDate = date.strftime('%Y/%m/%d %H:%M:%S')

            date = datetime.strptime(result['updatedAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
            updateDate = date.strftime('%Y/%m/%d %H:%M:%S')

            data = (pd.DataFrame(
                data=
                [{'id':result['id'],
                    '標題':result['title'],
                    '摘要':result['excerpt'],
                    '校名匿名':result['anonymousSchool'],
                    '匿名':result['anonymousDepartment'],
                    '發文時間':createDate,
                    '更新時間':updateDate,
                    '回應數量':result['commentCount'],
                    '愛心數量':result['likeCount'],
                    '標註':result['topics'],
                    }],
                columns=['id','標題','摘要','校名匿名','匿名','發文時間','更新時間','回應數量','愛心數量','標註']))
            list.append(data)
    return (lastId, list)


In [7]:
import pandas as pd
import requests
import re
from fake_useragent import UserAgent
import random
import time
from datetime import datetime
from requests.models import PreparedRequest

def fetchData(beforId: str):
    url = 'https://www.dcard.tw/service/api/v2/search/posts'
    params = {'query': beforId, "limit": 100}

    if beforId:
        time.sleep(3)
        params['before'] = beforId

    req = PreparedRequest()
    req.prepare_url(url, params)

    ua = UserAgent()

    resq = requests.get(req.url, headers = {
            "user-agent": ua.random
        })
    rejs = resq.json()

    return GenerateData(rejs)

In [8]:
result = []
lastId = "chifure"

while not result:
    (lastId, result) = fetchData(lastId)

df = pd.DataFrame()
df = pd.concat(result, ignore_index=True)

print(df.shape)
df

(3, 10)


,id,標題,摘要,校名匿名,匿名,發文時間,更新時間,回應數量,愛心數量,標註
0,237283174,美甲分享 canmake+chifure,開學的壓力真的需要美美指甲補充能量🥲，加上太想花錢 所以去Tomod’s 逛了一下！，這次分...,False,True,2021/10/22 13:14:03,2021/10/22 13:14:03,6,35,"[分享, canmake, 美甲, 指甲油, CHIFURE]"
1,235674291,分享#chifure 蜜粉餅 #打造偽素顏好氣色妝感#推,身為化妝新手兼懶人的我，想找一塊塗完防曬就可以直接上的粉餅，訴求不高，遮瑕力不用強，只想要提...,False,True,2021/04/01 07:30:24,2021/04/01 12:22:45,1,24,"[分享, 蜜粉餅]"
2,232994974,chifure口紅替芯推薦什麼色？,本人素顏黃皮，有去試用chifure口紅替芯，最後喜歡549和212（試到的色），但因為一開...,False,True,2020/02/01 06:52:43,2020/02/01 09:54:57,15,8,[口紅]
